<a href="figures/Nexus_logo_800px.png" target="_blank"><img src="figures/Nexus_logo_800px.png" 
width="150" border="10" /></a>

# Blue Brain Nexus - A knowledge graph for data-driven science - Part 2

## 5 - Exercises Part 2

### 5-0 Installation of Pyxus, import of relevant classes and client setup

Nexus exposes a RESTful interface over HTTP(S). To faciliate API interactions, the Nexus client Pyxus ([Pyxus on github](https://github.com/HumanBrainProject/pyxus)) can be used Pyxus to access and manage resources in Nexus.

**Execute the following code line to install Pyxus:**

In [ ]:
!pip install git+https://github.com/HumanBrainProject/pyxus@v0.1.2
!pip install requests
!pip install yaml

**Import relevant Pyxus classes:**

In [ ]:
import requests
import yaml
from pyxus.client import NexusClient
from pyxus.resources.entity import Organization
from pyxus.resources.entity import Domain
from pyxus.resources.entity import Schema
from pyxus.resources.entity import Instance

#### Grab a token here (use your Collab credentials to log in): 

https://bbp-nexus.epfl.ch/staging/v0/oauth2/authorize

Copy the token (the string of numbers and letters between the quotation marks) and paste it as the value of the 'token' variable below (please note that the token expires after 30 minutes, after which you have to grab a new token using the link above, replace your expired token below and rerun the Nexus client setup):

In [ ]:
token = "<PASTE YOUR TOKEN HERE>"

#### Setup your Nexus client by executing the code line below

In [ ]:
client = NexusClient(scheme="https", host="bbp-nexus.epfl.ch", prefix="staging/v0", alternative_namespace="https://bbp-nexus.epfl.ch", token=token)

### 5-1 Create a custom organization on Nexus using Pyxus

Top-level resources in Nexus are organizations (see diagram in Part 1). To set up your own organization, input a custom name and description for your organization below (e.g. use your first name as organization_name):

In [ ]:
organization_name = "<PUT YOUR ORGANIZATION NAME HERE>"
organization_description = "<PUT YOUR ORGANIZATION DESCRIPTION HERE>"

With the provided organization name and description, you can **create your organization** on Nexus:

In [ ]:
your_organization = Organization.create_new(organization_name, organization_description) 
client.organizations.create(your_organization)

You can access your organization resource through the link provided after executing the code below:

In [ ]:
your_organization = client.organizations.read(organization_name)
print(your_organization.data["@id"])

Check out your newly created organization in [Nexus Explorer](http://explorer.staging.nexus.ocp.bbp.epfl.ch/)

### 5-2 Create a custom domain on Nexus using Pyxus

Nested inside organizations on Nexus are domains. To set up your own domain, Input a custom name and decription for your domain below:

In [ ]:
domain_name = "<PUT YOUR DOMAIN NAME HERE>"
domain_description = "<PUT YOUR DOMAIN DESCRIPTION HERE>"

With the provided domain name and description, you can **create your domain** on Nexus:

In [ ]:
your_domain = Domain.create_new(organization_name, domain_name, domain_description) 
client.domains.create(your_domain)

You can access your domain resource through the link provided after executing the code below:

In [ ]:
your_domain = client.domains.read(organization_name, domain_name)
print(your_domain.data["@id"])

Check out your newly created domain in [Nexus Explorer](http://explorer.staging.nexus.ocp.bbp.epfl.ch/)

### 5-3 Create and publish schemas for Subject, Neuron and Dataset

Now that you have created your own organization and domain in Nexus, you can create the **schemas for Subject, Neuron and Dataset**. Copy-paste the three schemas from Part 1 as the value of the respective variable below:

In [ ]:
subject_schema = "<PASTE SUBJECT SCHEMA HERE>"

In [ ]:
neuron_schema = "<PASTE NEURON SCHEMA HERE>"

In [ ]:
dataset_schema = "<PASTE DATASET SCHEMA HERE>"

**Repeat** the following steps for the Subject, Neuron and Dataset schemas (adjust the schema_name and content variables accordingly):

-----

In [ ]:
schema_name = "subject"
schema_version = "v0.1.0"
content = subject_schema

Run the following code to create a schema and store it in Nexus:

In [ ]:
schema = Schema.create_new(organization=organization_name, domain=domain_name, schema='subject',version='v0.1.0', content=content)
client.schemas.create(schema)

Access the schema you have created:

In [ ]:
schema = client.schemas.read(organization=organization_name, domain=domain_name, schema=schema_name, version=schema_version)
print(subject.data["@id"])

To be able to submit data against your schema, it has to be published. To publish your schema, run the following code (note how the value of the key "published" in your schema changes):

In [ ]:
client.schemas.publish(entity=schema,publish=True)
print(subject.data["@id"])

Check out your newly created schemas in [Nexus Explorer](http://explorer.staging.nexus.ocp.bbp.epfl.ch/)

### 5-4 Create instances with prepared payloads

Now that the three schemas needed for our data are stored and published, we can validate our data against those schemas and store them in Nexus.

In [ ]:
subject_metadata = "<PASTE SUBJECT DATA HERE>"

In [ ]:
neuron_metadata = "<PASTE SUBJECT DATA HERE>"

In [ ]:
electrophysiology_metadata = "<PASTE ELECTROPHYSIOLOGY DATASET DATA HERE>"

**Repeat** the following steps for the Subject, Neuron, Morphology and Electrophysiology metadata (adjust the schema_name and content variables accordingly):

-----

In [ ]:
schema_name = "subject"
schema_version = "v0.1.0"
content = subject_schema

Run the following code to create an instance and store it in Nexus:

In [ ]:
instance = Instance.create_new(organization=organization_name, domain=domain_name, schema=schema_name, version=schema_version, content=subject_data)
client.instances.create(instance)

Retrieve all the instances you have created

In [ ]:
search_result = client.instances.list_by_schema(organization=organization_name, domain=domain_name, schema=schema_name, version=schema_version)
for result in search_result.results:
    print(result.result_id)
    instance_id = result.result_id

Check out your newly created data instances in [Nexus Explorer](http://explorer.staging.nexus.ocp.bbp.epfl.ch/)

### 5-5 Update the latest instance

After data has been stored in Nexus, it can be updated (e.g. if you want to provide a more detailed description or correct a typo). Take the payload of the last instance you have created and update its description. Paste the updated metadata as the value of the variable 'updated_instance_data" below:

In [ ]:
updated_instance_data = "<PASTE THE UPDATED INSTANCE DATA HERE>"

Using the updated payload, you can update your instance on Nexus:

In [ ]:
instance.data = updated_instance_data
client.instances.update(entity=instance)

List your instance and see the updated data:

In [ ]:
search_result = client.instances.list_by_schema(organization=organization_name, domain=domain_name, schema=schema_name, version=schema_version)
for result in search_result.results:
    print(result.result_id)
    instance_id = result.result_id

Check out your updated data instance in [Nexus Explorer](http://explorer.staging.nexus.ocp.bbp.epfl.ch/)

### 5-6 Attach binaries to the morphology and electrophysiology dataset instance

Nexus allows one to store metadata as well as file attachments. The following steps help you attach the provided morphology file (.ASC file format) and the electrophysiology recording file (.IBW file format). Please provide the correct filenames and nexus id's as the values of the variables below: 

In [ ]:
filename_morphology = "<PUT MORPHOLOGY FILENAME HERE>"
filename_electrophysiology = "<PUT ELECTROPHYSIOLOGY FILENAME HERE>"

morphology_dataset_id = "<NEXUS @ID OF THE MORPHOLOGY DATASET>"
electrophysiology_dataset_id = "<NEXUS @ID OF THE ELECTROPHYIOSLOGY DATASET>"

Run the following code to attach the morphology file to the morphology dataset instance on Nexus:

In [ ]:
url = "{}/attachment?rev=1".format(instance_id)
morphology = {'file': open(filename_morphology, 'rb')}
response = requests.put(url, files=morphology)

Run the following code to attach the electrophysiology file to the electrophysiology dataset instance on Nexus:

In [ ]:
url = "{}/attachment?rev=1".format(instance_id)
electrophysiology = {'file': open(filename_electrophysiology, 'rb')}
response = requests.put(url, files=electrophysiology)

Check out your datasets with attachments using [Nexus Explorer](http://explorer.staging.nexus.ocp.bbp.epfl.ch/)

### 5-7 Query Nexus to get all instance of type nsg:Neuron (spanning all the organizations)

In [58]:
query_filter = {
  "@context":   "https://bbp-nexus.epfl.ch/staging/v0/contexts/neurosciencegraph/core/data/v0.1.1",
  "filter": {
    "op": "eq",
    "path": "rdf:type",
    "value": "nsg:Neuron"
  }
}

In [60]:
response = requests.post("https://bbp-nexus.epfl.ch/staging/v0/queries", json=query_filter, allow_redirects=False)
print(response.text)

The request, and all future requests should be repeated using <a href="https://bbp-nexus.epfl.ch/staging/v0/queries/05021dfc-dd11-4b93-bdc8-2b44657ec4d4?from=0&size=10">this URI</a>.


## OPTIONAL 

### 5-8 Secure your organization by setting permissions

To restrict access to your organization, you can set specific access rights. Go through the steps outlined below to set permissions on your organization:

**Get your user reference and realm by running the following code:**

In [ ]:
response = requests.get("https://bbp-nexus.epfl.ch/staging/v0/oauth2/user", headers={"Authorization": "Bearer {}".format(token)})
if response.status_code < 400:
    response_text = yaml.load(response.text)["identities"]
    for identity in response_text:
        if identity["@type"] == "UserRef":
            user_reference = identity["sub"]
            realm = identity["realm"]
else:
     print("Status code: {}".format(response.status_code))

print(user_reference)
print(realm) 

**The following payload defines the permissions you want to set for your organization:**

In [ ]:
acls = {
    "acl": [
        {
            "permissions": [
            "read",
            "write",
            "own"
            ],
            "identity": {
            "realm": realm,
            "sub": user_reference,
            "@type": "UserRef"
            }
        }
    ]
}

**Use the defined acls to set permissions on your organization by running the code below:**

In [ ]:
response = requests.put(your_organization.data["@id"], json=acls)
if response.status_code >= 400:
    response_text = yaml.load(response.text)
    print(response_text.get('code'))
else:
    print("Status code: {}".format(response.status_code))

**Inspect the permissions on your organization**

In [ ]:
response = requests.get("https://bbp-nexus.epfl.ch/staging/v0/acls/kg/org1/experiment?parents=true", headers={"Authorization": "Bearer {}".format(token)})
if response.status_code < 400:
    response_text = yaml.load(response.text)["identities"]
    for identity in response_text:
        if identity["@type"] == "UserRef":
            user_reference = identity["sub"]
            realm = identity["realm"]
else:
     print("Status code: {}".format(response.status_code))